# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [12]:
import azureml.core
from azureml.core import Experiment
from azureml.core import Workspace

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.core import Dataset

from azureml.train.automl import AutoMLConfig

from azureml.pipeline.core import PipelineData, TrainingOutput

from azureml.widgets import RunDetails


import os 
import joblib

from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.webservice import LocalWebservice


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'wine_quality_automl'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)

dataset_name = 'wine_quality'
dataset = Dataset.get_by_name(workspace=ws, name=dataset_name) 

In [5]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.......................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=compute_target,
    task = "classification",
    training_data=dataset,
    label_column_name="quality",   
    path = project_folder,
    enable_early_stopping= True,
    featurization= 'auto',
    debug_log = "automl_errors.log",
    **automl_settings
)

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
wine_quality_automl,AutoML_dc713a40-c256-4baa-9a14-5b63a407c627,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_run, best_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

precision_score_macro 0.42566257446176875
f1_score_macro 0.36391636794717397
AUC_micro 0.9378293422131797
balanced_accuracy 0.35158197338208197
log_loss 0.7994044966596423
weighted_accuracy 0.7494437486245538
average_precision_score_macro 0.47970574595622023
AUC_weighted 0.8603987282424882
precision_score_weighted 0.6617065280349361
recall_score_micro 0.6904315196998124
accuracy 0.6904315196998124
average_precision_score_weighted 0.7657748526827827
f1_score_micro 0.6904315196998123
recall_score_macro 0.35158197338208197
AUC_macro 0.8679564551852779
matthews_correlation 0.49898467475174346
recall_score_weighted 0.6904315196998124
f1_score_weighted 0.6706363136479668
average_precision_score_micro 0.781687392922001
precision_score_micro 0.6904315196998124
norm_macro_recall 0.22189836805849836
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_dc713a40-c256-4baa-9a14-5b63a407c627_43/confusion_matrix
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_dc713a40-c256-4baa-9a14-

In [13]:
#TODO: Save the best model

os.makedirs('models', exist_ok=True)
joblib.dump(best_model, filename='./models/model.pkl')

model_name = best_run.properties['model_name']
description = "Best wine quality AutoML model"
model = remote_run.register_model(model_name = model_name, 
                                  description = description, 
                                  tags = None)



print(model_name)

AutoMLdc713a40c43


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [14]:
env = best_run.get_environment()
inference_config = InferenceConfig(
    environment=env,
    entry_script="inference.py",
)
deployment_config = LocalWebservice.deploy_configuration(port=6789)
service = Model.deploy(workspace=ws, 
    name='wine',
    models=[best_model], 
    inference_config=inference_config, 
    deployment_config=deployment_config, 
    overwrite=True)
)
service.wait_for_deployment(show_output=True)

ERROR:azureml._model_management._util:entry_script echo_score.py doesn't exist. entry_script should be path relative to current working directory



WebserviceException: WebserviceException:
	Message: entry_script echo_score.py doesn't exist. entry_script should be path relative to current working directory
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "entry_script echo_score.py doesn't exist. entry_script should be path relative to current working directory"
    }
}

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}
data = {
    "fixed acidity":7.3,
    "volatile acidity": 0.9,
    "citric acid": 0.1,
    "residual sugar": 1.9,
    "chlorides": 0.076,
    "free sulfur dioxide": 11.0, 
    "total sulfur dioxide": 34.0,
    "density": 0.9978,
    "pH": 3.49,
    "sulphates": 0.50,
    "alcohol": 9.4
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.text)
print(response.json())

In [ ]:
# register model
best_child = remote_run.get_best_child()
model = best_child.register_model(model_name='automl-model',model_path='outputs/model.pkl')
print(model.name, model.id, model.version, sep=' ') 

# deploy full instance
service_name = "wine-quality-prediction"
env = best_run.get_environment()
print(env)

inference_config = InferenceConfig(
    environment=env,
    entry_script="inference.py",
)

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 0.5, 
    memory_gb=1, 
    auth_enabled=True
)
web_service = Model.deploy(
    workspace=ws, 
    name=service_name,
    models=[model], 
    inference_config=inference_config, 
    deployment_config=aci_config, 
    overwrite=True
)
web_service.wait_for_deployment(show_output=True)

In [ ]:
uri = web_service.scoring_uri
api_key = 'tbd'
headers = {"Content-Type": "application/json",  'Authorization':('Bearer '+ api_key)}
data = {
    "fixed acidity":7.3,
    "volatile acidity": 0.9,
    "citric acid": 0.1,
    "residual sugar": 1.9,
    "chlorides": 0.076,
    "free sulfur dioxide": 11.0, 
    "total sulfur dioxide": 34.0,
    "density": 0.9978,
    "pH": 3.49,
    "sulphates": 0.50,
    "alcohol": 9.4
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(web_service.get_logs())
web_service.delete()
model.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
